# Covid-19 DB MS SQL Server Connection with pyodbc & sqlalchemy

In [1]:
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings("ignore")

In [2]:
#  Creating the data df for SQL Server

df_covid = pd.read_csv(r"C:\Users\serda\Desktop\SD RECOVERY\DATA SCIENCE\DATASETS\COVID-19 Activity.csv", low_memory=False)

df_covid.head()

,PEOPLE_POSITIVE_CASES_COUNT,COUNTY_NAME,PROVINCE_STATE_NAME,REPORT_DATE,CONTINENT_NAME,DATA_SOURCE_NAME,PEOPLE_DEATH_NEW_COUNT,COUNTY_FIPS_NUMBER,COUNTRY_ALPHA_3_CODE,COUNTRY_SHORT_NAME,COUNTRY_ALPHA_2_CODE,PEOPLE_POSITIVE_NEW_CASES_COUNT,PEOPLE_DEATH_COUNT
0,0,Lee,Alabama,2020-01-22,America,New York Times,0,1081.0,USA,United States,US,0,0
1,0,Lee,Alabama,2020-01-23,America,New York Times,0,1081.0,USA,United States,US,0,0
2,0,Lee,Alabama,2020-01-24,America,New York Times,0,1081.0,USA,United States,US,0,0
3,0,Lee,Alabama,2020-01-25,America,New York Times,0,1081.0,USA,United States,US,0,0
4,0,Lee,Alabama,2020-01-26,America,New York Times,0,1081.0,USA,United States,US,0,0


In [3]:
# Converting the column names lowercase

df_covid.rename(columns=lambda x: x.lower(), inplace=True)

df_covid.head()

,people_positive_cases_count,county_name,province_state_name,report_date,continent_name,data_source_name,people_death_new_count,county_fips_number,country_alpha_3_code,country_short_name,country_alpha_2_code,people_positive_new_cases_count,people_death_count
0,0,Lee,Alabama,2020-01-22,America,New York Times,0,1081.0,USA,United States,US,0,0
1,0,Lee,Alabama,2020-01-23,America,New York Times,0,1081.0,USA,United States,US,0,0
2,0,Lee,Alabama,2020-01-24,America,New York Times,0,1081.0,USA,United States,US,0,0
3,0,Lee,Alabama,2020-01-25,America,New York Times,0,1081.0,USA,United States,US,0,0
4,0,Lee,Alabama,2020-01-26,America,New York Times,0,1081.0,USA,United States,US,0,0


In [4]:
# Listing SQL drivers

pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [6]:
#  Creating engine for MS SQL Server for pyodbc and sqlalchemy

server = 'localhost'
dbname = 'master'
driver = 'ODBC+Driver+17+for+SQL+Server'

engine_pyodbc = pyodbc.connect('driver={'+driver.replace("+", " ")+'};'+'server='+server+';database='+dbname+';trusted_connection=yes')
engine_alchemy = create_engine(f'mssql+pyodbc://{server}/{dbname}?trusted_connection=yes&driver={driver}')

In [7]:
engine_pyodbc.autocommit = True

In [8]:
# Creating user defined query function

def excecute_query(engine, query:str, print_rows=False):
    """
    This function send the query to SQL Server

    engine:
        engine
    query: str
        query string
    print_rows: Boolean
        print query results, default False
    """
    if type(engine) == pyodbc.Connection:
        crs = engine.cursor()
        if print_rows:
            for row in crs.execute(query).fetchall():
                print(row)
        else:
            crs.execute(query)
            print(f'pyodbc query executed successfully.')
        crs.close()

    elif str(type(engine)).__contains__('sqlalchemy'):
        with engine.connect() as conn:
            if print_rows:
                for row in conn.execute(text(query)).fetchall():
                    print(row)
            else:
                conn.execute(text(query))
                print(f'sqlalchemy query executed successfully.')
            conn.commit()
    else:
        print('Please define a valid conneciton.')

In [9]:
excecute_query(engine_pyodbc, """CREATE DATABASE Covid19;""")

pyodbc query executed successfully.


In [10]:
excecute_query(engine_alchemy, """USE Covid19;""")

sqlalchemy query executed successfully.


In [11]:
excecute_query(engine_alchemy, 'CREATE SCHEMA covid;')

sqlalchemy query executed successfully.


In [12]:
# inserting df to sql server (to_sql method cannot be used in pyodbc)

df_covid.to_sql('covid', engine_alchemy, 'covid', index=False, if_exists='append')

53

In [13]:
excecute_query(engine_alchemy,
               """SELECT country_short_name,
               SUM(people_positive_new_cases_count) positive_cases
               FROM covid.covid
               GROUP BY country_short_name
               ORDER BY positive_cases DESC;""",
               print_rows=True)

('United States', 81237970)
('India', 43075864)
('Brazil', 30433042)
('France', 27829894)
('Germany', 24798067)
('United Kingdom', 22038340)
('Russia', 17909924)
('Korea, South', 17237878)
('Italy', 16409183)
('Turkey', 15030321)
('Spain', 11833457)
('Vietnam', 10644700)
('Argentina', 9072230)
('Netherlands', 8131213)
('Japan', 7846274)
('Iran', 7221121)
('Colombia', 6092403)
('Indonesia', 6046467)
('Poland', 5995674)
('Australia', 5927528)
('Mexico', 5738769)
('Ukraine', 5040518)
('Malaysia', 4445897)
('Thailand', 4250949)
('Austria', 4135344)
('Israel', 4074112)
('Belgium', 4056448)
('Czechia', 3905101)
('Portugal', 3853800)
('South Africa', 3785398)
('Canada', 3762254)
('Philippines', 3685403)
('Switzerland', 3619598)
('Peru', 3564558)
('Chile', 3556333)
('Greece', 3317425)
('Denmark', 3115721)
('Romania', 2892989)
('Slovakia', 2528216)
('Sweden', 2500917)
('Iraq', 2324982)
('China', 2154960)
('Serbia', 2005832)
('Bangladesh', 1952674)
('Hungary', 1901017)
('Jordan', 1695745)
('Geor

In [15]:
excecute_query(engine_alchemy,"SELECT * FROM covid.covid WHERE report_date BETWEEN '2022-01-01' AND '2022-04-30'",
               print_rows=True)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



(7101, None, None, '2022-01-05', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 314, 200)
(14165, None, None, '2022-04-17', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 0, 219)
(14629, None, None, '2022-04-29', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 0, 220)
(14428, None, None, '2022-04-23', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 96, 220)
(13690, None, None, '2022-03-10', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 0, 216)
(13936, None, None, '2022-03-18', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 15, 217)
(13690, None, None, '2022-03-15', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 0, 216)
(14428, None, None, '2022-04-25', 'America', 'JHU CSSE Global Timeseries', 0, None, 'GRD', 'Grenada', 'GD', 0, 220)
(13982, None, None, '2022-03-29', 'America', 'JHU CSSE Global Timeser

In [17]:
excecute_query(engine_alchemy,"SELECT count( *) FROM covid.covid",
               print_rows=True)

(2952310,)
